# charged_words
One goal of *Moviegoer* is identifying scenes or conversations with a high level of drama. We can look for specific words or phrases to quantify drama.

In [1]:
from subtitle_dataframes_io import *
nlp = spacy.load('en')

In [2]:
subs = pysrt.open('../subtitles/plus_one_2019.srt')
subtitle_df = generate_base_subtitle_df(subs)
subtitle_df = generate_subtitle_features(subtitle_df)
subtitle_df['cleaned_text'] = subtitle_df['concat_sep_text'].map(clean_line)
sentences = partition_sentences(remove_blanks(subtitle_df['cleaned_text'].tolist()), nlp)
subtitle_indices = tie_sentence_subtitle_indices(sentences, subtitle_df)
sentence_df = pd.DataFrame(list(zip(sentences, subtitle_indices)), columns=['sentence', 'subtitle_indices'])

## Profanity
The easiest way to measure drama is by identifying and counting profanity. We can define a list of profanities for which to search. Then we'll break the sentence into lemma, so that we can find derivations in the vein of "hecked", "heckin'", "hecking", "hecky", etc.

This is defined as a function *profanity()* in *phrases_io.py* and is used to find profanity at the scene- and film-level. These can be compared to find scenes with higher profanity than the film average.

In [3]:
sentence_df[800:830]

,sentence,subtitle_indices
800,I can be tough.,[741]
801,But you're always there...,[742]
802,It's so fucking hot.,[743]
803,always ready to catch me when I fall.,[744]
804,Always my best friend.,[745]
805,I need to pee.,[746]
806,If we can just sit on the couch and eat Chines...,"[747, 748, 749]"
807,I'm gonna make a run for it.,[750]
808,No.,[751]
809,Wait.,[751]


In [4]:
sentence = sentence_df.iloc[814].sentence
sentence

"He's fucking talking about babies."

In [5]:
sent_doc = nlp(sentence)
profanity_count = 0

profanities = ['fuck', 'fucking', 'fuckin', 'fucked', 'shit', 'shitty', 'bullshit', 'ass', 'dumbass', 'bitch', 'asshole', 'tit', 'cunt', 'goddamn', 'damn', 'dammit', 'cock', 'cocksucker', 'dick']
for word in sent_doc:
    if word.lemma_.lower() in profanities:
        profanity_count += 1

profanity_count


1